In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=8, bias=True)
  (relu): ReLU()
)


In [4]:
train_models(env, models, episodes=10, steps=2000, print_every=500)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: 2.9799999999999898
----STEP 500 rewards----
Model 0: -4.056816560536163
----STEP 1000 rewards----
Model 0: -6.852380311352675
----STEP 1500 rewards----
Model 0: -14.32956684974684
------EPISODE 0 rewards------
Model 0: -23.367599054424716
----STEP 0 rewards----
Model 0: 0.9799999999999898
----STEP 500 rewards----
Model 0: -4.89277152994768
----STEP 1000 rewards----
Model 0: -13.449284067307957
----STEP 1500 rewards----
Model 0: -20.61902221594852
------EPISODE 1 rewards------
Model 0: -28.384617515516965
----STEP 0 rewards----
Model 0: 0.9799999999999898
----LEARNING BEGINS----
----STEP 500 rewards----
Model 0: -6.909520472772385
----STEP 1000 rewards----
Model 0: -12.493925030583597
----STEP 1500 rewards----
Model 0: -20.640249599029204
------EPISODE 2 rewards------
Model 0: -20.583394867194727
----STEP 0 rewards----
Model 0: 1.9799999999999898
----STEP 500 rewards----
Model 0: 1.8325856596887604
----STEP 1000 rewards----
Model 0: -7.0444291

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_11")